# 프로그램 붙이기

프로그램 `g`와 프로그램 `f`를 결합한 `(g . f)`도 프로그램이다. 이 프로그램은 `input`에 대해 다음처럼 `f`가 먼저 계산하고, 그 결과를 받아서 `g`가 처리한다.

```haskell
g (f input)
```

그런데 만약 `f`가 무한히 끝나지 않는 프로그램이라면 어떨까? 혹은 `f`가 생성한 결과가 너무 커서 메모리에 올릴 수 없다면 `g`는 어떻게 그 결과를 처리할 수 있을까?

함수형 언어는 이러한 문제에 대해 `f`와 `g`를 엄격한 동기화하여 실행시키는 방법을 제공한다. 즉, `g`가 시작되어 `f`로부터 값을 요구하는 경우에만 `f`가 실행되며, `f`는 `g`가 요구한 만큼의 값을 계산한 다음에는 중지된다. `g`가 다시 값을 요구하면 `f`는 아까 동작을 재개하여 또 필요한 만큼의 결과만 계산하고 중지된다. `f`가 무한히 계속 동작하게 구현되었다 하더라도, `g`가 종료되면 `f`도 함께 종료된다. **종료 조건을 분리시킨다는 건 강력한 모듈화를 가능하게 한다**.

이처럼 `f`를 가능한 최소로 실행/평가하는 방법을 "지연 평가(lazy evaluation)"이라고 부른다. 지연 평가로 인해 프로그램을 generator와 selector로 모듈화할 수 있다. generator는 후보값들을 생성하고, selector는 문제의 답을 찾는다. generator입장에서는 무한히 값을 생성할 수 있다. 다른 언어/환경에서도 이런 식의 지연 평가를 구현할 수는 있지만, 함수형 언어는 모든 함수 호출에 대해 일관되게 이 방식을 적용하고 있어서 프로그램의 어떤 부분이든 이 같은 방법의 모듈화를 적용할 수 있다. 지연 평가는 매우 강력한 도구여서 함수형 프로그래머라면 반드시 갖춰야 할 도구이다.


## 뉴튼-랩슨 방법

'[뉴튼-랩슨](https://en.wikipedia.org/wiki/Newton%27s_method)' 수치 알고리즘을 통해 지연 연산(lazy evaluation)이 어떤 형태로 도움을 줄 수 있는지 살펴본다.
뉴튼-랩슨 방법에 따라, 어떤 수 n의 제곱근을 구하려면 아래의 식을 반복적용하여 점점 더 정확한 근사값을 구하면 된다.

$a_{i+1} = (a_i + n / a_i ) / 2 $

매우 간단하다. C/Java로 구현한다면 아래와 같다.([출처:프린스턴대 자바 강의](http://introcs.cs.princeton.edu/java/21function/))

```java
public static double sqrt(double c, double EPSILON) {
    if (c < 0) return Double.NaN;
    double t = c;
    while (Math.abs(t - c/t) > EPSILON*t)
        t = (c/t + t) / 2.0;
    return t;
}
```

간단하기는 하지만 더 작은 조각들로 나누기는 어려워보인다. 지연 평가를 이용하면 더 작은 조각들로 나눌 수 있다.

뉴튼 방법은 근사값을 점점 더 정확하게 만드는, 즉 근사값을 계속 만들어내는 방법이므로 근사값의 리스트를 반환하는 것이 적절해보인다. 이전 근사값으로부터 좀더 정확한 다음 근사값을 만드는 함수는 $a_{i+1} = (a_i + n / a_i ) / 2$ 수식에 따라 다음과 같이 정의할 수 있다.

In [10]:
next n x = (x + n / x) / 2

인자를 하나만 적용한 `next n`은 근사값을 다음 근사값으로 매핑하는 함수이다. 이 함수를 `f`라고 하면 근사값의 리스트는 `[a0, f a0, f(f a0), f(f(f a0)), ... ]`이므로 이러한 패턴을 함수로 정의할 수 있다.

```haskell
repeat f a = a : repeat f (f a)
```

하지만 Haskell에는 이미 이러한 함수가 `iterate`라는 이름으로 정의되어 있다.

In [11]:
take 10 (iterate (+1) 0)

[0,1,2,3,4,5,6,7,8,9]

이제 근사값의 리스트는 다음과 같이 계산할 수 있다.

```haskell
iterate (next n) a0
```

`iterate`는 **무한히 긴** 리스트를 만들어내지만, 그건 가능성을 의미하는 것일뿐 실제로는 프로그램이 필요로 하는 만큼만 만들어낸다. 

이제 제곱근의 근사값을 구하기 위해 `within`이라는 함수를 정의할 것이다. 이 함수는 리스트의 값들을 따라가면서 이웃한 두 값이 주어진 오차범위에 들어가면 그 값을 반환한다.

In [12]:
within eps (a:b:rest) 
 | abs (a-b) <= eps     = b
 | otherwise            = within eps (b:rest)

이 조각 함수들을 붙이면 다음과 같은 `sqrt`함수를 얻을 수 있다.

In [13]:
sqrt a0 eps n = within eps (iterate (next n) a0)

In [14]:
sqrt 1 0.001 2

1.4142135623746899

각 조각들을 다른 형태로 조합할 수도 있다. 예를 들어, `within`이 가지는 문제점(원래 값이 매우 작은 경우 오차를 비교하기 어렵다)을 개선한 `relative` (두 값의 크기 비율이 1과 가까운지 확인) 함수를 사용할 수도 있다.

In [15]:
relative eps (a:b:rest)
 | abs (a/b - 1) <= eps   = b
 | otherwise              = relative eps (b:rest)

이 비교 함수를 사용하는 새 함수를 만든다면 `within`을 `relative`로 바꾸기만 하면 된다.

In [16]:
relativesqrt a0 eps n = relative eps (iterate (next n) a0)

In [17]:
relativesqrt 1 0.001 2

1.4142135623746899

근사값의 리스트를 생성하는 부분(`next`나 `iterate` 등)을 다시 정의할 필요가 없다.